# 根据已有的数据库转换为ORM模型
- 在实际开发中，有些时候可能数据库已经存在了，如果要用django来开发一个网站，读取的是之前已经存在于数据库中的数据。根据旧的的数据库生成对应的ORM模型，需要以下几个步骤：
- 在此之前，先把SQL文件导入navicat软件，先新建一个数据库，在数据库下的表右击，然后执行SQL文件即可将已有的SQL文件导入到数据库中
    - 1.django给我们提供了一个inspectdb的命令，可以自动将数据表转换为ORM模型。想要使用inspectdb表生产模型。首先需要在settings.py中配置好数据库相关信息，否则找不到数据库
    - 2.配置好以后，在终端进入到django项目所在的路径，运行python manage.py inspectdb > models.py，即可将转换得到的模型导入models.py文件，>表示重定向。如果只执行pyhton manage.py inspectdb，得到的模型只会在终端显示。
    - 3.修正模型：新生成的ORM模型有些地方可能不太适合使用，比如模型的名字，表之间的关系等，那么一下选项需要重新配置一下
        - 模型名：自动生成的模型，是根据表的名字生成的，如果名字不合适，改动即可。
        - 模型所属APP：根据自己的需要，将相应的APP放在对应的APP中，便于管理。之后要将APP注册到settings.py文件
        - 模型外键的引用：将所有使用ForeignKey的地方，模型引用都改成字符串，这样不会产生模型顺序的问题。另外，如果引用的模型已经移动到其他的APP中了，那么还要加上这个APP的前缀
        - 删掉managed=False，让django有责任管理模型：将Meta下的managed=False删掉，如果保留，那么以后这个模型有任何的修改，使用migrate都不会映射到数据库中
        - 表名：切记不要修改表的名字，不然映射到数据库中，会发生找不到对应表的错误
        - 表关联：如果表与表是多对多关系，则会生成一个中间表，此时需要把它注释掉。同时在子表中引入ManyToManyField("",db_table),db_table表示这个中间表生成的名字。一般来说需要重命名以下
    - 4.执行命令Python manage.py makemigrations生成初始化的迁移脚本，方便后面通过ORM来管理表。然后执行命令python manage.py migrate APP名字 --fake-initial（APP名字就是你将模型放到的那些APP的名字，注意只能一个一个的进行fake），因为如果不使用--fake-initial，那么将会迁移脚本映射到数据库中。这时候迁移脚本会新创建表，而这个表已经存在，所以肯定会报错。此时我们只要将migrations文件下的第一个文件状态修改为已经映射，而不真正执行，下次再migrate的时候就会忽略它。
    - 5.将django的核心表映射到数据库中，django中还有一些核心的表也是需要创建的，不然有些功能无法使用。比如auth相关表。如果这个数据库之前就是使用django开发的，那么这些表就已经存在了，可以不用创建它们了。如果不是，那么应该使用migrate命令将django中的核心模型映射到数据库中